In [1]:
from datasets import load_dataset
import torch

trainFile="Data/s288c.fasta.train.txt"
testText="CAA GTG ACC AGA ATG ATC ACC GGT GTT CAA CAA"

In [2]:
dataset=load_dataset('text', data_files=trainFile,split='train')

Using custom data configuration default-f555aa9781812fb7
Reusing dataset text (/home/lu/.cache/huggingface/datasets/text/default-f555aa9781812fb7/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


In [3]:
batch_size = 1000
all_texts = [dataset[i : i + batch_size]["text"] for i in range(0, len(dataset), batch_size)]

In [4]:
def batch_iterator():
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]["text"]

In [5]:
from transformers import BertTokenizer, BertModel

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


In [6]:
new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=5000)

In [7]:
new_tokenizer.save_pretrained("S288C_TOKENIZER")

('S288C_TOKENIZER/tokenizer_config.json',
 'S288C_TOKENIZER/special_tokens_map.json',
 'S288C_TOKENIZER/vocab.txt',
 'S288C_TOKENIZER/added_tokens.json',
 'S288C_TOKENIZER/tokenizer.json')